# Chains in LangChain

### Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import pandas as pd

df = pd.read_csv('Data.csv')
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [2]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]
llm_model = "gemma-3-27b-it" # "gemini-2.0-flash-lite"

llm = ChatGoogleGenerativeAI(
    model=llm_model,
    temperature=0.9,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give your answer in one short paragraph"
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "Queen Size Sheet Set"
chain.run(product)

C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_14540\1694080553.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_14540\1694080553.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Here are a few options, ranging in style, for a company selling Queen Size Sheet Sets:\n\n**Cozy Queen Linens** is a strong choice – it\'s descriptive, evokes comfort, and clearly states the focus. Other good options include **Queen Dream Sleep**, **The Queen\'s Bed**, or **Simply Queen Sheets** if you want something more straightforward. Ultimately, the "best" name depends on your brand\'s overall aesthetic (luxury, budget-friendly, modern, etc.), but prioritizing clarity and a feeling of comfort is key for bedding.'

In [ ]:
## LangChain 0.3

chain = prompt | llm

product = "Queen Size Sheet Set"

chain.invoke({"product": product})

AIMessage(content='Here are a few options, ranging in style, for a company selling Queen Size Sheet Sets:\n\n**Cozy Queen Linens** is a strong choice – it\'s descriptive, evokes comfort, and clearly states the focus. Other good options include **Queen Dream Sheets**, **The Queen\'s Bed**, or **Simply Queen Sheets**. If you want to emphasize quality, **Royal Queen Collection** or **Queen Comforts** could work. Ultimately, the "best" name depends on your brand\'s overall aesthetic and target audience, but prioritizing clarity and a feeling of comfort is key.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1baecfd9-13a3-4953-87fc-eebdd8322e1d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 119, 'total_tokens': 144, 'input_token_details': {'cache_read': 0}})

## Simple Sequential Chain

In [6]:
from langchain.chains import SimpleSequentialChain

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Give your answer in two short paragraph"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Here are a few name ideas, broken down by approach, with my top picks at the end:

**Descriptive & Functional:** "Queen Comforts," "Queen Size Linens," "The Queen Sheet Co.," "Restful Queen," or "Simply Queen Sheets" clearly state what you sell. **Elegant & Evocative:** "Serene Slumber," "Royal Rest," "Dream Weaver Linens," or "The Velvet Queen" suggest a luxurious experience. **Modern & Minimalist:** "Q Sleep," "Loom Queen," or "The Sheet Edit" are short, memorable, and appeal to a contemporary aesthetic.

**My top recommendations are:** **"Queen Comforts"** – it's straightforward, inviting, and highlights the benefit. Or, **"Serene Slumber"** – if you're aiming for a more premium, relaxing brand image. Ultimately, the "best" name depends on your target audience and the overall brand you want to create. Check for trademark availability before settling on a name!
Here are 20-word descriptions for both "Queen Comforts" & "Serene Slumber," 

'Here are 20-word descriptions for both "Queen Comforts" & "Serene Slumber," reflecting the provided brand ideas:\n\n**Queen Comforts:** Luxurious, high-quality queen size sheets & bedding designed for ultimate comfort & restful sleep. Affordable indulgence for your nightly retreat.\n\n**Serene Slumber:** Experience blissful, restorative sleep with our elegantly crafted queen linens. Indulge in luxurious comfort & wake up feeling refreshed & renewed.'

In [ ]:
simple_chain = first_prompt | llm | (lambda response: {"company_name": response.content}) | second_prompt | llm

simple_chain.invoke({"product": product}).content

'Here are a few 20-word descriptions, tailored to the name options:\n\n**For "Queen Comforts":**\n\nLuxurious queen-size sheets & bedding designed for ultimate comfort. Experience restful nights and wake refreshed with Queen Comforts\' quality linens.\n\n**For "Serene Slumber":**\n\nIndulge in serene nights with Serene Slumber\'s exquisitely crafted queen bedding. Experience luxurious comfort and wake up feeling truly rejuvenated.'

## Sequential Chain

In [13]:
from langchain.chains import SequentialChain

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short and polite follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.',
 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit or old batch due to its inferiority to store-bought versions.",
 'followup_message': 'Here are a few options, ranging from slightly more formal to slightly more casual, all polite and appropriate:\n\n**Option 1 (More Formal):**\n\n> Bonjour,\n\n> Nous vous remercions pour votre ret

In [21]:
from langchain_core.output_parsers import StrOutputParser

# Create the chain using the pipe operator
translate_chain = first_prompt | llm | StrOutputParser()
summarize_chain = second_prompt | llm | StrOutputParser()
language_chain = third_prompt | llm | StrOutputParser()
followup_message = fourth_prompt | llm | StrOutputParser()

# Define the complete workflow
def process_review(review):
    # Run translation and store result
    english_review = translate_chain.invoke({"Review": review})
    
    # Run summarization using the translated review
    summary = summarize_chain.invoke({"English_Review": english_review})
    
    # Determine language
    language = language_chain.invoke({"Review": review})
    
    # Generate followup
    followup_result = followup_message.invoke({"summary": summary, "language": language})
    
    return {
        "English_Review": english_review,
        "summary": summary,
        "language": language,
        "followup_message": followup_result
    }

result = process_review(df.Review[5])
result

{'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.',
 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit or old batch due to its inferiority to store-bought versions.",
 'language': "The language is **French**.\n\nHere's a quick translation to confirm:\n\n* **Je trouve le goût médiocre.** - I find the taste mediocre.\n* **La mousse ne tient pas, c'est bizarre.** - The foam doesn't hold, it's strange.\n* **J'achète les mêmes dans le commerce et le goût est bien meilleur...** - I buy the same ones in stores and the taste is much better...\n* **Vieux lot ou cont

In [15]:
from langchain_core.runnables import RunnablePassthrough

# Create the chain using the pipe operator
translate_chain = first_prompt | llm | RunnablePassthrough()
summarize_chain = second_prompt | llm | RunnablePassthrough()
language_chain = third_prompt | llm | RunnablePassthrough()

# Define the complete workflow
def process_review(review):
    # Run translation and store result
    english_review = translate_chain.invoke({"Review": review})
    
    # Run summarization using the translated review
    summary = summarize_chain.invoke({"English_Review": english_review})
    
    # Determine language
    language = language_chain.invoke({"Review": review})
    
    # Generate followup
    followup_message = fourth_prompt | llm | RunnablePassthrough()
    followup_result = followup_message.invoke({"summary": summary, "language": language})
    
    return {
        "English_Review": english_review,
        "summary": summary,
        "language": language,
        "followup_message": followup_result
    }

result = process_review(df.Review[5])
result

{'English_Review': AIMessage(content='Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0a6ef3a8-54fb-45d9-81f9-83b7f509be69-0', usage_metadata={'input_tokens': 54, 'output_tokens': 94, 'total_tokens': 148, 'input_token_details': {'cache_read': 0}}),
 'summary': AIMessage(content='This reviewer is dissatisfied with the product, finding the taste subpar, the foam strange, and suspects it may be an old or counterfeit batch due to its inferior quality compared to store-bought versions.', add

In [20]:
# Create the chain using the pipe operator
translate_chain = first_prompt | llm | StrOutputParser()
summarize_chain = second_prompt | llm | StrOutputParser()
language_chain = third_prompt | llm | StrOutputParser()
followup_chain = fourth_prompt | llm | StrOutputParser()

# Define the complete workflow using `|`
pipeline = (
    RunnablePassthrough.assign(Review=lambda x: x["Review"])  # Extract Review
    | {
        "English_Review": translate_chain,  # Translate review to English
        "language": language_chain         # Detect language
    }
    | {
        "English_Review": RunnablePassthrough(),  # Ensure it’s passed forward
        "summary": summarize_chain,  # Summarize the English review
        "language": RunnablePassthrough()  # Pass language forward
    }
    | {
        "summary": RunnablePassthrough(),       # Summarize the translated review
        "language": RunnablePassthrough(),  # Pass language forward
        "followup_message": followup_chain  # Generate follow-up message
    }
)

# Run the pipeline
result = pipeline.invoke({"Review": df.Review[5]})

print(result)

{'summary': {'English_Review': {'English_Review': 'Here\'s the translation of the review:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?" \n\nEssentially, the reviewer is saying the product tastes bad, the foam is weird, and it doesn\'t compare to the ones they buy elsewhere, leading them to suspect it might be old stock or a fake.', 'language': "The language is **French**.\n\nHere's a quick translation:\n\n* **Je trouve le goût médiocre.** - I find the taste mediocre.\n* **La mousse ne tient pas, c'est bizarre.** - The foam doesn't hold, it's strange.\n* **J'achète les mêmes dans le commerce et le goût est bien meilleur...** - I buy the same ones in stores and the taste is much better...\n* **Vieux lot ou contrefaçon !?** - Old batch or counterfeit!?"}, 'summary': "This reviewer was disappointed with the product's poor taste and texture, suspecting it may be a counterfeit o

## Router Chain

In [6]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know. Start your answer with: As a physics professor\

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question. Start your answer with: As a mathematician\

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements. Start your answer with: As a historian

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. Start your answer with: As a successful computer scientist

Here is a question:
{input}"""

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising \
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:70: SyntaxWarning: invalid escape sequence '\ '
<>:70: SyntaxWarning: invalid escape sequence '\ '
C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_14540\3497777481.py:70: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [7]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\IdeaPad\AppData\Local\Temp\ipykernel_14540\691140245.py:30: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [8]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'As a physics professor, black body radiation refers to the electromagnetic radiation emitted by an object solely due to its temperature. \n\nHere\'s the breakdown:\n\n* **Black Body:** An idealized physical body that *absorbs all* incident electromagnetic radiation, regardless of frequency or angle. Because it absorbs everything, it\'s perfectly black at low temperatures.\n* **Radiation:** When heated, this "perfect absorber" also becomes a perfect *emitter*. The spectrum of light it emits (the intensity of each color/wavelength) depends *only* on its temperature, not its material.\n* **Key Features:** The emitted radiation has a continuous spectrum.  The higher the temperature, the more radiation is emitted *and* the shorter the wavelengths at which the peak emission occurs (this is described by Wien\'s Displacement Law).  The total energy radiated is proportional to the fourth power of the temperature (Stefan-Boltzmann Law).\n\n**Think of it like this:** A hot piece of metal glows. 

In [9]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As a mathematician, even the simplest questions deserve a thorough approach! Let\'s break down "2 + 2" into its fundamental components.\n\n**Component 1: Understanding Addition**\n\nAddition, denoted by the "+" symbol, is a mathematical operation that combines two or more quantities (numbers) to find their total. It represents the process of joining sets.\n\n**Component 2: Understanding the Number 2**\n\nThe number 2 represents a quantity of two units. We can visualize this as having two objects – for example, two apples: 🍎🍎.\n\n**Component 3: Applying Addition to the Problem**\n\n"2 + 2" means we are combining two groups, each containing two units.  Let\'s visualize this:\n\n🍎🍎 + 🍎🍎\n\n**Component 4: Counting the Total**\n\nNow, we count the total number of apples: 🍎🍎🍎🍎.  There are four apples.\n\n**Putting it Together**\n\nTherefore, 2 + 2 = 4. \n\nWhile seemingly trivial, this demonstrates the core principles of mathematical reasoning – breaking down a problem, understanding its co

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

### LangChain 0.3

In [ ]:
from operator import itemgetter
from typing import Literal

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langgraph.graph import END, START, StateGraph
from typing_extensions import TypedDict

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0.9,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("system", physics_template),
        ("human", "{input}"),
    ]
)
prompt_2 = ChatPromptTemplate.from_messages(
    [
        ("system", math_template),
        ("human", "{input}"),
    ]
)
prompt_3 = ChatPromptTemplate.from_messages(
    [
        ("system", "Just answer the questions. Start your answer with: as a lay person"),
        ("human", "{input}"),
    ]
)

# Construct the chains we will route to. These format the input query
# into the respective prompt, run it through a chat model, and cast
# the result to a string.
chain_1 = prompt_1 | llm | StrOutputParser()
chain_2 = prompt_2 | llm | StrOutputParser()
chain_3 = prompt_3 | llm | StrOutputParser()

# Next: define the chain that selects which branch to route to.
# Here we will take advantage of tool-calling features to force
# the output to select one of two desired branches.
route_system = "Route the user's query to either the physics or math expert or lay person."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{input}"),
    ]
)

# Define schema for output:
class RouteQuery(TypedDict):
    """Route query to destination expert."""

    destination: Literal["physics", "math", "lay person"]

route_chain = route_prompt | llm.with_structured_output(RouteQuery)

# For LangGraph, we will define the state of the graph to hold the query,
# destination, and final answer.
class State(TypedDict):
    query: str
    destination: RouteQuery
    answer: str


# We define functions for each node, including routing the query:
async def route_query(state: State, config: RunnableConfig):
    destination = await route_chain.ainvoke(state["query"], config)
    # return {"destination": destination}
    return destination


# And one node for each prompt
async def prompt_1(state: State, config: RunnableConfig):
    return {"answer": await chain_1.ainvoke(state["query"], config)}

async def prompt_2(state: State, config: RunnableConfig):
    return {"answer": await chain_2.ainvoke(state["query"], config)}

async def prompt_3(state: State, config: RunnableConfig):
    return {"answer": await chain_3.ainvoke(state["query"], config)}

# We then define logic that selects the prompt based on the classification
def select_node(state: State) -> Literal["prompt_1", "prompt_2", "prompt_3"]:
    # print(state["destination"])
    if state["destination"] == "physics":
        return "prompt_1"
    elif state["destination"] == "math":
        return "prompt_2"
    elif state["destination"] == "lay person":
        return "prompt_3"
    else:
        return "prompt_3"

# Finally, assemble the multi-prompt chain. This is a sequence of two steps:
# 1) Select "animal" or "vegetable" via the route_chain, and collect the answer
# alongside the input query.
# 2) Route the input query to chain_1 or chain_2, based on the
# selection.
graph = StateGraph(State)
graph.add_node("route_query", route_query)
graph.add_node("prompt_1", prompt_1)
graph.add_node("prompt_2", prompt_2)
graph.add_node("prompt_3", prompt_3)

graph.add_edge(START, "route_query")
graph.add_conditional_edges("route_query", select_node)
graph.add_edge("prompt_1", END)
graph.add_edge("prompt_2", END)
graph.add_edge("prompt_3", END)
app = graph.compile()


c:\Users\IdeaPad\anaconda3\envs\pytorch\Lib\site-packages\langchain_google_genai\chat_models.py:1295: UserWarning: ChatGoogleGenerativeAI.with_structured_output with dict schema has changed recently to align with behavior of other LangChain chat models. More context: https://github.com/langchain-ai/langchain-google/pull/772
  warnings.warn(


In [ ]:
# from IPython.display import Image

# Image(app.get_graph().draw_mermaid_png())

In [33]:
state = await app.ainvoke({"query": "What is black body radiation?"})
print(state["destination"])
print(state["answer"])

physics
As a physics professor, black body radiation is the electromagnetic radiation emitted by a perfect black body. A perfect black body is an idealized object that absorbs all electromagnetic radiation that falls on it, regardless of frequency or angle. Because it absorbs all radiation, it also emits radiation at all frequencies, depending only on its temperature. The spectrum of this emitted radiation is described by Planck's law, which shows that the intensity and frequency distribution of the radiation depend on the object's temperature.


In [32]:
state = await app.ainvoke({"query": "what is 2 + 2"})
print(state["destination"])
print(state["answer"])

math
As a mathematician, I can certainly answer that question!

Here's how we solve it:

*   **Understanding the problem:** We are asked to add the number 2 to the number 2.
*   **Applying the operation:** Addition is the process of combining quantities.
*   **Calculation:** 2 + 2 = 4

Therefore, the answer is $\boxed{4}$
